In [28]:
import re, os, sys, unicodedata
import pandas as pd
import numpy as np
import py_stringmatching as sm
import codecs

In [ ]:
path = '/Users/galvanize/infosys/cip/data/nc_data'
filename0, filename1 = 'nc0.csv', 'nc1.csv'
filepath0, filepath1 = os.path.join(path, filename0), os.path.join(path, filename1)

In [77]:
df0 = pd.read_csv(filepath0, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
df1 = pd.read_csv(filepath1, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
ROW_BOUNDARY = 1000 #separation btw first and second files
df = pd.concat([df0.sample(ROW_BOUNDARY), df1.sample(ROW_BOUNDARY)]).reset_index(drop=True)

answers = df['recid']
df = df.drop(['recid'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   givenname  2000 non-null   object
 1   surname    2000 non-null   object
 2   suburb     1999 non-null   object
 3   postcode   1999 non-null   object
dtypes: object(4)
memory usage: 62.6+ KB


In [78]:
for col in df.columns:
    df.loc[:,col] = df[col].apply(lambda x: str(x)).str.replace('[^A-Za-z0-9]+', '', regex=True)

In [79]:
tok = sm.QgramTokenizer(qval=3, return_set=True, padding=False)

In [80]:
for col in df.columns:
    df.loc[:,f'{col}_qgrams'] = df[col].apply(lambda x: sorted(tok.tokenize(x)))
df.head()

,givenname,surname,suburb,postcode,givenname_qgrams,surname_qgrams,suburb_qgrams,postcode_qgrams
0,edwin,wilson,highlands,28741,"[dwi, edw, win]","[ils, lso, son, wil]","[and, ghl, hig, hla, igh, lan, nds]","[287, 741, 874]"
1,michelle,fryman,oxford,27565,"[che, ell, hel, ich, lle, mic]","[fry, man, rym, yma]","[for, ord, oxf, xfo]","[275, 565, 756]"
2,rebecca,dully,monroe,28110,"[bec, cca, ebe, ecc, reb]","[dul, lly, ull]","[mon, nro, onr, roe]","[110, 281, 811]"
3,finnard,caldwell,greensboro,27406,"[ard, fin, inn, nar, nna]","[ald, cal, dwe, ell, ldw, wel]","[bor, een, ens, gre, nsb, oro, ree, sbo]","[274, 406, 740]"
4,rachel,wells,tarboro,27886,"[ach, che, hel, rac]","[ell, lls, wel]","[arb, bor, oro, rbo, tar]","[278, 788, 886]"


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   givenname         2000 non-null   object
 1   surname           2000 non-null   object
 2   suburb            2000 non-null   object
 3   postcode          2000 non-null   object
 4   givenname_qgrams  2000 non-null   object
 5   surname_qgrams    2000 non-null   object
 6   suburb_qgrams     2000 non-null   object
 7   postcode_qgrams   2000 non-null   object
dtypes: object(8)
memory usage: 125.1+ KB


In [82]:
all_suburb_qgrams = set().union(*df['suburb_qgrams'].values)
all_givenname_qgrams = set().union(*df['givenname_qgrams'].values)
all_surname_qgrams = set().union(*df['surname_qgrams'].values)
all_postcode_qgrams = set().union(*df['postcode_qgrams'].values)
len(all_suburb_qgrams), len(all_givenname_qgrams), len(all_surname_qgrams), len(all_postcode_qgrams)

(1340, 1384, 2276, 606)

In [139]:
from collections import defaultdict
from itertools import combinations
from more_itertools import partition

In [146]:
postcode_dict = dict(zip(df.index, df.postcode_qgrams))
inverted_postcode_dict = defaultdict(list)
for key, values in postcode_dict.items():
    for value in values:
        inverted_postcode_dict[value].append(key)

rows_to_compare_postcode = set()
for key in inverted_postcode_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_postcode_dict[key], 2))
    rows_to_compare_postcode.update(set(combos))

len(rows_to_compare_postcode)

64661

In [147]:
len(rows_to_compare_postcode)

64661

In [149]:
suburb_dict = dict(zip(df.index, df.suburb_qgrams))
inverted_suburb_dict = defaultdict(list)
for key, values in suburb_dict.items():
    for value in values:
        inverted_suburb_dict[value].append(key)

rows_to_compare_suburb = set()
for key in inverted_suburb_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_suburb_dict[key], 2))
    rows_to_compare_suburb.update(set(combos))
len(rows_to_compare_suburb)

98958

In [150]:
surname_dict = dict(zip(df.index, df.surname_qgrams))
inverted_surname_dict = defaultdict(list)
for key, values in surname_dict.items():
    for value in values:
        inverted_surname_dict[value].append(key)

rows_to_compare_surname = set()
for key in inverted_surname_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_surname_dict[key], 2))
    rows_to_compare_surname.update(set(combos))
len(rows_to_compare_surname)

23179

In [151]:
givenname_dict = dict(zip(df.index, df.givenname_qgrams))
inverted_givenname_dict = defaultdict(list)
for key, values in givenname_dict.items():
    for value in values:
        inverted_givenname_dict[value].append(key)

rows_to_compare_givenname = set()
for key in inverted_givenname_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_givenname_dict[key], 2))
    rows_to_compare_givenname.update(set(combos))
len(rows_to_compare_givenname)

23520

In [154]:
final = (set().union(rows_to_compare_givenname,
                     rows_to_compare_surname,
                     rows_to_compare_suburb,
                     rows_to_compare_postcode
                    )
        )

len(final)

182916

In [155]:
ROW_BOUNDARY**2

1000000

In [156]:
len(final)/ROW_BOUNDARY**2

0.182916